In [1]:
import numpy as np
import pandas as pd
import os, glob, random, matplotlib, cv2
import matplotlib.patches as mplpatches
from collections import Counter
import pandas as pd
from xml.etree import ElementTree as ET
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers

In [ ]:
!pip install xmltodict

In [7]:
import xmltodict

# Load the data <a class="anchor" id="loaddata"></a>

In [9]:
annotations_path = "/kaggle/input/face-mask-detection/annotations"
images_path = "/kaggle/input/face-mask-detection/images"

CHANNELS = 3
IMAGE_SIZE = 224
INPUT_SIZE = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
TRAIN_SET_PRCNT = 0.70

In [10]:
def get_annotation(annotation_file):
    
    objects = {
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
            "name":[],    
            "file":[],
            "width":[],
            "height":[],
           }

    tree = ET.parse(annotation_file)

    for el in tree.iter():
        if 'size' in el.tag:
            for attr in list(el):
                if 'width' in attr.tag: 
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))    

        if 'object' in el.tag:
            for attr in list(el):

                if 'name' in attr.tag:
                    name = attr.text
                    if name == "mask_weared_incorrect":
                        name == "with_mask"
                    
                    if name == "with_mask":
                        name = 1
                    else:
                        name = 0
                    objects['name']+=[name]
                    objects['width']+=[width]
                    objects['height']+=[height] 
                    objects['file']+=[annotation_file.split('/')[-1][0:-4]] 

                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            objects['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            objects['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            objects['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            objects['ymax']+=[ymax]     
    
    return objects


In [ ]:
image_files = [
    f for f in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, f))
]
annot_files = [
    f for f in os.listdir(annotations_path) if os.path.isfile(os.path.join(annotations_path, f))
]

image_files.sort()
annot_files.sort()

images, targets = [], []
vals = []

for i in range(0, len(annot_files)):
    annot = get_annotation(os.path.join(annotations_path, annot_files[i]))

    top_left_x, top_left_y = annot['xmax'], annot['ymax']
    bottom_right_x, bottom_right_y = annot['xmin'], annot['ymin']

    image = keras.utils.load_img(
        os.path.join(images_path, image_files[i]),
    )
    (w, h) = image.size[:2]
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))

    for i in range(0, len(top_left_x)):
        images.append(keras.utils.img_to_array(image))
        targets.append(
            (                
                float(top_left_x[i]) / w,
                float(top_left_y[i]) / h,
                float(bottom_right_x[i]) / w,
                float(bottom_right_y[i]) / h,
                
            )
        )
        vals.append(annot["name"])
        
    
t = []
    
for i in range(len(vals)):
    for j in range(len(vals[i])):
        t.append(vals[i][j])
    
'''
    t.append(vals[i])
    
    one = 0
    zero = 0
    for j in range(len(targets[i])):
        if targets[i][j] == 1:
            one += 1
        else:
            zero += 1
    
    if one > zero:
        t.append(1)
    else:
        t.append(0)
    '''

    
(x_train), (y_train) = (
    np.asarray(images[: int(len(images) * TRAIN_SET_PRCNT)]),
    np.asarray(t[: int(len(t) * TRAIN_SET_PRCNT)]).astype('float32'),
)

(x_test), (y_test) = (
    np.asarray(images[int(len(images) * TRAIN_SET_PRCNT) :]),
    np.asarray(t[int(len(t) * TRAIN_SET_PRCNT) :]).astype('float32'),
)

imgs = []

for i in range(len(x_train)):
    x = x_train[i]
    top_left_x, top_left_y = int(targets[i][0]* 224), int(targets[i][1] * 224)
    bottom_right_x, bottom_right_y = int(targets[i][2] * 224), int(targets[i][3] * 224)
    img = x[bottom_right_y:top_left_y, bottom_right_x:top_left_x]
    shape = img.shape
    xDiff = 244-shape[0]
    xLeft = xDiff//2
    xRight = xDiff-xLeft

    yDiff = 244-shape[1]
    yLeft = yDiff//2
    yRight = yDiff - yLeft
    
    imgs.append(np.pad(img,((xLeft,xRight),(yLeft,yRight),(5,5)), mode='constant'))
    
for i in range(len(x_test)):
    x = x_test[i]
    top_left_x, top_left_y = int(targets[i + len(x_train)][0]* 224), int(targets[i + len(x_train)][1] * 224)
    bottom_right_x, bottom_right_y = int(targets[i + len(x_train)][2] * 224), int(targets[i + len(x_train)][3] * 224)
    
    img = x[bottom_right_y:top_left_y, bottom_right_x:top_left_x]
    shape = img.shape
    xDiff = 244-shape[0]
    xLeft = xDiff//2
    xRight = xDiff-xLeft

    yDiff = 244-shape[1]
    yLeft = yDiff//2
    yRight = yDiff - yLeft
    
    imgs.append(np.pad(img,((xLeft,xRight),(yLeft,yRight),(5,5)), mode='constant'))

imgs = np.asarray(imgs)

(x_train) = np.asarray(imgs[: int(len(imgs) * TRAIN_SET_PRCNT)])


(x_test) = np.asarray(imgs[int(len(imgs) * TRAIN_SET_PRCNT) :])

'''
x = x_train[0]
fig, ax1 = plt.subplots(1, figsize=(3,3))
top_left_x, top_left_y = int(y_train[i][0]* 224), int(y_train[i][1] * 224)
bottom_right_x, bottom_right_y = int(y_train[i][2] * 224), int(y_train[i][3] * 224)
box_predicted = [top_left_x, top_left_y, bottom_right_x, bottom_right_y]

im = x[bottom_right_y:top_left_y, bottom_right_x:top_left_x]
print(box_predicted)
# Display the image
ax1.imshow(im.astype("uint8"))
'''

In [ ]:
input_image = x_train[0]
fig, ax1 = plt.subplots(1, figsize=(15, 15))
im = input_image
ax1.imshow(im.astype("uint8"))


input_image = cv2.resize(
    input_image, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_AREA
)
(h, w) = (im).shape[0:2]

for i in range(0,3):
    top_left_x, top_left_y = int(y_train[i][0] * 224), int(y_train[i][1] * 224)
    bottom_right_x, bottom_right_y = int(y_train[i][2] * 224), int(y_train[i][3] * 224)
    box_predicted = [top_left_x, top_left_y, bottom_right_x, bottom_right_y]
    rect = mplpatches.Rectangle(
                            (top_left_x, top_left_y),
                            bottom_right_x - top_left_x,
                            bottom_right_y - top_left_y,
                            facecolor="none",
                            edgecolor="red",
                            linewidth=1,
                        )
    print(box_predicted)
    ax1.add_patch(rect)

plt.show()

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
SEED = 42
DROPOUT = 0.1
NUM_HEADS = 8
PADDING = "VALID"
EPSILON = 1e-6
EPOCHS = 10
PATCH_SIZE = 32
NUM_LAYERS = 10
WEIGHT_DECAY = 1e-4
BATCH_SIZE = 4
MLP_DIM = 128
PROJECTION_DIM = 64
LR = 1e-6

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def get_config(self):
        config = super().get_config().copy()
        config.update(
            {
                "input_shape": input_shape,
                "patch_size": patch_size,
                "num_patches": num_patches,
                "projection_dim": projection_dim,
                "num_heads": num_heads,
                "transformer_units": transformer_units,
                "transformer_layers": transformer_layers,
                "mlp_head_units": mlp_head_units,
            }
        )
        return config

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding=PADDING,
        )
        return tf.reshape(patches, [batch_size, -1, patches.shape[-1]])


In [ ]:
random_num = random.randint(0, len(x_train)-1)

plt.figure(figsize=(4, 4))
plt.imshow(x_train[random_num]/255.)
plt.axis("off")

patches = Patches(PATCH_SIZE)(tf.convert_to_tensor([x_train[random_num]]))
print(f"Image size: {IMAGE_SIZE} X {IMAGE_SIZE}")
print(f"Patch size: {PATCH_SIZE} X {PATCH_SIZE}")
print(f"{patches.shape[1]} patches per image \n{patches.shape[-1]} elements per patch")


n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(6, 6))

for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (PATCH_SIZE, PATCH_SIZE, CHANNELS))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")


In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def get_config(self):
        config = super().get_config().copy()
        config.update(
            {
                "input_shape": input_shape,
                "patch_size": patch_size,
                "num_patches": num_patches,
                "projection_dim": projection_dim,
                "num_heads": num_heads,
                "transformer_units": transformer_units,
                "transformer_layers": transformer_layers,
                "mlp_head_units": mlp_head_units,
            }
        )
        return config

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded


In [ ]:
def create_vit_object_detector(
                                input_shape,
                                patch_size,
                                num_patches,
                                projection_dim,
                                num_heads,
                                transformer_units,
                                transformer_layers,
                                mlp_head_units):
    
    
    inputs = layers.Input(shape=input_shape)
    patches = Patches(PATCH_SIZE)(inputs)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=EPSILON)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=DROPOUT
        )(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=EPSILON)(x2)
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=DROPOUT)
        encoded_patches = layers.Add()([x3, x2])

    representation = layers.LayerNormalization(epsilon=EPSILON)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(DROPOUT*3)(representation)
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=DROPOUT*3)

    bounding_box = layers.Dense(1)(features)

    return keras.Model(inputs=inputs, outputs=bounding_box)


In [ ]:
from tensorflow.python.client import device_lib
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')), "\n")

print(device_lib.list_local_devices())

tf.debugging.set_log_device_placement(True)
tf.random.set_seed(SEED)

In [ ]:
from keras import backend as K
from tensorflow.keras.optimizers import SGD

class BalancedSparseCategoricalAccuracy(keras.metrics.SparseCategoricalAccuracy):
    def __init__(self, name='balanced_sparse_categorical_accuracy', dtype=None):
        super().__init__(name, dtype=dtype)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_flat = y_true
        if y_true.shape.ndims == y_pred.shape.ndims:
            y_flat = tf.squeeze(y_flat, axis=[-1])
        y_true_int = tf.cast(y_flat, tf.int32)

        cls_counts = tf.math.bincount(y_true_int)
        cls_counts = tf.math.reciprocal_no_nan(tf.cast(cls_counts, self.dtype))
        weight = tf.gather(cls_counts, y_true_int)
        return super().update_state(y_true, y_pred, sample_weight=weight)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def train(model, learning_rate, weight_decay, batch_size, num_epochs):

    optimizer = tfa.optim.SGD(learning_rate, weight_decay)

    
    model.compile(
                    optimizer=optimizer, 
                    loss=['binary_crossentropy'],
                    metrics=['acc',f1_m,precision_m, recall_m]
                  )
    
    checkpoint_filepath = "logs/"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True,
    )
    class_weight = {0: 0.51, 1: 0.48}
    tr = model.fit(
                            x=x_train,
                            y=y_train,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            validation_split=0.25,
                            class_weight= class_weight,
                            callbacks=[
                                        checkpoint_callback,
                                        keras.callbacks.EarlyStopping(monitor="val_loss", patience=10),
                                      ],
                         )

    return tr


num_patches = (IMAGE_SIZE // PATCH_SIZE) ** 2

transformer_units = [
                        PROJECTION_DIM * 2,
                        PROJECTION_DIM,
                    ]

transformer_layers = 6
mlp_head_units = [2048, 1024, 512, 64, 32]


history = []
num_patches = (IMAGE_SIZE // PATCH_SIZE) ** 2

vit_object_detector = create_vit_object_detector(
                                                    INPUT_SIZE,
                                                    PATCH_SIZE,
                                                    num_patches,
                                                    PROJECTION_DIM,
                                                    NUM_HEADS,
                                                    transformer_units,
                                                    transformer_layers,
                                                    mlp_head_units,
                                                )

## Visualize ViT model <a class="anchor" id="visualize"></a>

In [ ]:
vit_object_detector.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(vit_object_detector, to_file='model.png')

## Training the model <a class="anchor" id="fitting"></a>

In [ ]:
# Train model
for i in range(5):
    print(f"Test {i + 1}")
    train(vit_object_detector, LR, WEIGHT_DECAY, BATCH_SIZE, 10)

# Evaluate <a class="anchor" id="future"></a>

In [ ]:
print("Evaluate on test data")
test_loss, test_acc, f1, precision, recall = vit_object_detector.evaluate(x_test, y_test, batch_size=BATCH_SIZE)
print("test loss: ", test_loss)
print("test acc: ", test_acc)
print("f1: ", f1)
print("precision: ", precision)
print("recall: ", recall)

acc_ones = 0
acc_zeroes = 0
pre_ones = 0
pre_zeroes = 0
correct = 0

for i in y_test:
    if i == 1:
        acc_ones += 1
    else:
        acc_zeroes += 1

for index, i in enumerate(vit_object_detector.predict(x_test)):
    if i > 0.5:
        pre_ones += 1
        if y_test[index] == 1:
            correct += 1
    else:
        if y_test[index] == 0:
            correct += 1
            print(True)
        pre_zeroes += 1
        

print(correct/len(y_test))        
print(acc_ones, acc_zeroes)
print(pre_ones, pre_zeroes)

con = tf.math.confusion_matrix(labels=y_test, predictions=vit_object_detector.predict(x_test))
print(con)